In [9]:
from subgrounds.subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
import pandas as pd
import requests as r
import sys
sys.path.append('../')
import defillama_utils as dfl
import optimism_pool_tvls as tvls
from datetime import timedelta
import time
import os

### Data Sources
- Messari Subgraphs: [https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum](https://thegraph.com/hosted-service/subgraph/messari/curve-finance-ethereum)

In [10]:
chain_list = ['arbitrum','aurora','avalanche','ethereum',\
        'fantom','gnosis','harmony','moonbeam','optimism','polygon']
trailing_num_days = 180
start_time = pd.to_datetime("today") - timedelta(days = trailing_num_days-1)
start_time = int( time.mktime(start_time.timetuple()) )

# def get_messari_sg_pool_snapshots(slug, chains, min_ts, max_ts):
get_curve = tvls.get_messari_sg_pool_snapshots('curve-finance',chain_list,start_time)


curve-finance
arbitrum
aurora
avalanche
ethereum
fantom
gnosis
harmony
moonbeam
optimism
polygon
['id', 'timestamp', 'totalValueLockedUSD', 'dailyVolumeUSD', 'rewardTokenEmissionsUSD', 'protocol_id', 'protocol_name', 'protocol_slug', 'protocol_network', 'pool_id', 'pool_name', 'pool_symbol']


### Export and Show

In [14]:
if not os.path.exists('csv_outputs'):
    os.makedirs('csv_outputs')
get_curve.to_csv('csv_outputs/curve_snapshot_data.csv')
# display(get_curve)

In [15]:
# group by the specified columns
grouped = get_curve.sort_values(by='timestamp', ascending=False)\
                  .groupby(['protocol_network', 'pool_id', 'pool_name', 'pool_symbol'])

# perform the desired aggregations
aggregated = grouped.agg({'totalValueLockedUSD': 'first'
                          , 'dailyVolumeUSD': 'mean'
                          , 'rewardTokenEmissionsUSD':'sum'
                          ,'timestamp':'nunique'
                          })

aggregated = aggregated.rename(columns={'totalValueLockedUSD':'latest_tvl_usd'
                                        ,'dailyVolumeUSD':'avg_daily_volume_usd'
                                        ,'timestamp':'days_of_trading'})

# reset the index
aggregated = aggregated.reset_index()

aggregated = aggregated[aggregated['pool_id'] != 0]

display(aggregated.sort_values(by='days_of_trading',ascending=False))
aggregated.to_csv('csv_outputs/curve_aggregated_data.csv')

,protocol_network,pool_id,pool_name,pool_symbol,latest_tvl_usd,avg_daily_volume_usd,rewardTokenEmissionsUSD,days_of_trading
41,AVALANCHE,0x1da20ac34187b2d9c74f729b85acb225d3341b25,Curve.fi Factory Plain Pool: YUSD Pool,YUSDCRV-f,4.508767e+08,3.719112e+07,0.0,180
605,MATIC,0xda43bfd7ecc6835aa6f1761ced30b986a574c0d2,Curve.fi Factory USD Metapool: deBridge-USDC,deUSDC-3P3CRV-f,1.729052e+07,1.075352e+06,0.0,180
60,AVALANCHE,0x7f90122bf0700f9e7e1f688fe926940e8839f353,Curve.fi avDAI/avUSDC/avUSDT,av3CRV,1.190984e+07,5.895172e+06,0.0,180
131,FANTOM,0x9dc516a18775d492c9f061211c8a3fdcd476558d,Curve.fi Factory Plain Pool: 4Pool,4POOL-f,2.522311e+05,2.139742e+03,0.0,180
635,MOONBEAM,0xc6e37086d09ec2048f151d11cdb9f9bbbdb7d685,Curve.fi Factory Plain Pool: stdot,DOTSTDOT-f,0.000000e+00,1.295913e+04,0.0,180
...,...,...,...,...,...,...,...,...
574,MATIC,0xb6f8e5ddf3b03e475e8eee609231397de23b87d9,Curve.fi Factory USD Metapool: Stable Corinthian,Corinthian3CRV-f,8.463376e+05,2.082697e-02,0.0,1
575,MATIC,0xb70aa4ad7fd5352eb2ebdd052de17f2f29971ecb,Curve.fi Factory Plain Pool: 2gold (CACHE),2gold CACH-f,0.000000e+00,0.000000e+00,0.0,1
124,FANTOM,0x7e4143ed63635e5f53f0272c307eac4a5238037a,Curve.fi Factory Plain Pool: TestAssent,APUSDTEST-f,1.755945e+00,1.653663e-02,0.0,1
445,MAINNET,0xf4a3cca34470b5ba21e2bb1ed365acf68b4d4598,Curve.fi Factory Crypto Pool: bll3,bll3c-f,0.000000e+00,1.019127e-01,0.0,1
